In [1]:
import numpy as np
import pandas as pd
from datetime import *
from time import sleep
import os
import requests


In [2]:
API_key_static = "" #Add your own key
API_key_real_time = "" #Add your own key
API_Platsupplag = "" #Add your own key
API_Planing = "" #Add your own key


# Get the stations

## Functions

In [4]:
def search_station(station_search: str):
    '''Do a search on the API to get the siteID of a station.
    Warning : 
    - max 10 results so the desired output may be not displayed
    - need at least 3 characters to have a result'''
    API_Platsupplag = "bdcd8c42c08b46578e734afb7b54bd8e"
    url = "https://api.sl.se/api2/typeahead.json?key=" + \
        API_Platsupplag+"&searchstring="+station_search
    test = requests.get(url)
    return pd.json_normalize(test.json()["ResponseData"])


In [8]:
df_res = search_station("Solna")
df_res


,Name,SiteId,Type,X,Y,Products
0,Solna centrum (Solna),9305,Station,17997555,59358677,None
1,Solna centrum norra (Solna),3472,Station,17999775,59361697,None
2,Solna (Solna),9509,Station,18011866,59364313,None
3,Solna station (Solna),9509,Station,18011866,59364313,None
4,Solna strand (Solna),9326,Station,17974381,59353499,None
5,Solna Business Park (Solna),5119,Station,17979720,59359477,None
6,Solna strand - Huvudstaleden (Solna),9328,Station,17975486,59352249,None
7,Solna tenniscenter (Solna),3402,Station,17986732,59375783,None
8,Solnahallen (Solna),3475,Station,17985222,59358020,None
9,Solna kyrka (Solna),3409,Station,18024289,59353822,None


## Results

In [10]:
dico_stationID = {"T-Centralen": 9001,
                  "Tekniska högskolan": 9204,
                  "Solna Centrum": 9305,
                  "Solna Bus": 3472,
                  "Fridhemsplan": 9115,
                  "Globen": 9168,
                  "Bergshamra": 9202,
                  "Bergshamra bus": 3431}
dico_station_usage = {"T-Centralen": ["Metros"],
                      "Tekniska högskolan": ["Metros", "Buses"],
                      "Solna Centrum": ["Metros", "Buses", "Trams"],
                      "Solna Bus": ["Buses"],
                      "Fridhemsplan": ["Metros", "Buses"],
                      "Globen": ["Metros", "Trams"],
                      "Bergshamra": ["Metros"],
                      "Bergshamra bus": ["Buses"]}
dico_station_time_window = {"T-Centralen": 30,
                            "Tekniska högskolan": 20,
                            "Solna Centrum": 20,
                            "Solna Bus": 20,
                            "Fridhemsplan": 20,
                            "Globen": 20,
                            "Bergshamra": 30,
                            "Bergshamra bus": 25}


# Collecting real-time data

## Functions

In [27]:
def valid_hour():
    target_hour = [7, 8, 16, 17]
    return datetime.now().hour in target_hour


In [6]:
# Loop
API_key_real_time = "3185727dca7a4cc09c65dee39cee35be"
day = str(datetime.today().date())
timewindow = "10"
i = 0
while True:
    if valid_hour():
        df = pd.DataFrame(columns=['GroupOfLine', 'TransportMode', 'LineNumber', 'Destination',
                                   'JourneyDirection', 'StopAreaName', 'StopAreaNumber', 'StopPointNumber',
                                   'StopPointDesignation', 'TimeTabledDateTime', 'ExpectedDateTime',
                                   'DisplayTime', 'JourneyNumber', 'Deviations', 'RequestTime', 'Station'])
        for station in dico_stationID.keys():
            siteid = dico_stationID[station]
            url = "http://api.sl.se/api2/realtimedeparturesv4.json?key=" + API_key_real_time + "&siteid=" + \
                str(siteid) + "&timewindow=" + \
                str(dico_station_time_window[station]
                    ) + "&Train=False&Ship=False"
            test = requests.get(url)
            time_request = datetime.now()
            for mode in dico_station_usage[station]:
                try:
                    df_append = pd.DataFrame(
                        test.json()["ResponseData"][mode]).sort_values("ExpectedDateTime")
                    df_append["RequestTime"] = time_request
                    df_append["Station"] = station
                    df = pd.concat([df, df_append])
                except:
                    print(station, mode, datetime.now())
          # Save data
        if not os.path.isfile('data/data_collection_'+day+'.csv'):
            df.to_csv('data/data_collection_'+day+'.csv',
                      header='column_names', index=False, encoding='utf-8')
        else:  # else it exists so append without writing the header
            df.to_csv('data/data_collection_'+day+'.csv', mode='a',
                      header=False, index=False, encoding='utf-8')
        sleep(15)
    else:
        sleep(10)
        day = str(datetime.today().date())


T-Centralen Metros 2023-05-10 16:22:17.508603
Tekniska högskolan Metros 2023-05-10 16:22:18.790251
Tekniska högskolan Buses 2023-05-10 16:22:18.791210
Solna Centrum Metros 2023-05-10 16:22:18.829319
Solna Centrum Buses 2023-05-10 16:22:18.829319
Solna Centrum Trams 2023-05-10 16:22:18.830315
Solna Bus Buses 2023-05-10 16:22:18.861636
Friedhemsplan Metros 2023-05-10 16:22:18.925279
Friedhemsplan Buses 2023-05-10 16:22:18.925279
Globen Metros 2023-05-10 16:22:21.036903
Globen Trams 2023-05-10 16:22:21.038003
Bergshamra Metros 2023-05-10 16:22:21.841845
Bergshamra bus Buses 2023-05-10 16:22:22.340029


KeyboardInterrupt: 

# Get SL proposition

In [55]:
journey_single =[["Tekniska högskolan","Fridhemsplan"],["Tekniska högskolan", "Solna Centrum"],["Globen", "Solna Centrum"]]
journey_list = np.concatenate([[[a[0],a[1]],[a[1],a[0]]] for a in (journey_single)],axis=0)
df_planner = pd.DataFrame(columns=['StartStation', 'EndStation', 'Lines', 'DepartureStations',
                            'ArrivalStations','DepartureTime','ArrivalTime','TransitionTime','RequestTime'])
for journey in journey_list:
    start = dico_stationID[journey[0]]
    end = dico_stationID[journey[1]]
    url = "http://api.sl.se/api2/TravelplannerV3_1/trip.csv?key="+API_Planing+"&Lang=en&originId="+str(start)+"&destId="+str(end)
    test = requests.get(url)
    request_time = datetime.now()
    try:
        answer = test.json()["Trip"]
        for i in range(len(answer)):
            trans_times =[]
            dep_times =[]
            dep_stations =[]
            arr_times =[]
            arr_stations =[]
            lines=[]
            for j in range(len(answer[i]["LegList"]["Leg"])):
                answer_leg = answer[i]["LegList"]["Leg"][j]
                origin = answer_leg["Origin"]
                destination = answer_leg["Destination"]
                mode = answer[i]["LegList"]["Leg"][j]["type"]
                dep_time = datetime.strptime(origin["time"],"%H:%M:%S")
                arr_time = datetime.strptime(destination["time"],"%H:%M:%S")
                dep_stations.append(origin["name"])
                arr_stations.append(destination["name"])
                if mode=="WALK":
                    walk_time = (arr_time-dep_time).total_seconds()//60
                    trans_times.append(walk_time)
                    dep_times.append(0)
                    arr_times.append(0)
                else:
                    trans_times.append(0)
                    dep_times.append(dep_time)
                    arr_times.append(arr_time)
                    lines.append(int(answer_leg["name"].split(" ")[-1]))
                departure_time = answer_leg["Destination"]
            df_add = pd.DataFrame([[journey[0],journey[1],lines,dep_stations,arr_stations,dep_times,arr_times,trans_times,request_time]],columns=df_planner.columns)
            df_planner = pd.concat([df_planner,df_add],ignore_index=True)
    except:
        print("Fail from "+journey[0]+" to "+journey[1])


# Collecting real-time data and SL proposition at the same time

In [ ]:
# Loop
API_key_real_time = "" #Add your own key
day = str(datetime.today().date())
timewindow = "10"
journey_single =[["Tekniska högskolan","Fridhemsplan"],["Tekniska högskolan", "Solna Centrum"],["Globen", "Solna Centrum"]]
journey_list = np.concatenate([[[a[0],a[1]],[a[1],a[0]]] for a in (journey_single)],axis=0)
i = 0
while True:
    if valid_hour():
        df = pd.DataFrame(columns=['GroupOfLine', 'TransportMode', 'LineNumber', 'Destination',
                                   'JourneyDirection', 'StopAreaName', 'StopAreaNumber', 'StopPointNumber',
                                   'StopPointDesignation', 'TimeTabledDateTime', 'ExpectedDateTime',
                                   'DisplayTime', 'JourneyNumber', 'Deviations', 'RequestTime', 'Station'])
        for station in dico_stationID.keys():
            siteid = dico_stationID[station]
            url = "http://api.sl.se/api2/realtimedeparturesv4.json?key=" + API_key_real_time + "&siteid=" + \
                str(siteid) + "&timewindow=" + \
                str(dico_station_time_window[station]
                    ) + "&Train=False&Ship=False"
            test = requests.get(url)
            time_request = datetime.now()
            for mode in dico_station_usage[station]:
                try:
                    df_append = pd.DataFrame(
                        test.json()["ResponseData"][mode]).sort_values("ExpectedDateTime")
                    df_append["RequestTime"] = time_request
                    df_append["Station"] = station
                    df = pd.concat([df, df_append])
                except:
                    print(station, mode, datetime.now())
          # Save data
        if not os.path.isfile('data/data_collection_'+day+'.csv'):
            df.to_csv('data/data_collection_'+day+'.csv',
                      header='column_names', index=False, encoding='utf-8')
        else:  # else it exists so append without writing the header
            df.to_csv('data/data_collection_'+day+'.csv', mode='a',
                      header=False, index=False, encoding='utf-8')
        df_planner = pd.DataFrame(columns=['StartStation', 'EndStation', 'Lines', 'DepartureStations',
                                   'ArrivalStations','DepartureTime','ArrivalTime','TransitionTime','RequestTime'])
        for journey in journey_list:
            start = dico_stationID[journey[0]]
            end = dico_stationID[journey[1]]
            url = "http://api.sl.se/api2/TravelplannerV3_1/trip.csv?key="+API_Planing+"&Lang=en&originId="+str(start)+"&destId="+str(end)
            test = requests.get(url)
            request_time = datetime.now()
            try:
                answer = test.json()["Trip"]
                for i in range(len(answer)):
                    trans_times =[]
                    dep_times =[]
                    dep_stations =[]
                    arr_times =[]
                    arr_stations =[]
                    lines=[]
                    for j in range(len(answer[i]["LegList"]["Leg"])):
                        answer_leg = answer[i]["LegList"]["Leg"][j]
                        origin = answer_leg["Origin"]
                        destination = answer_leg["Destination"]
                        mode = answer[i]["LegList"]["Leg"][j]["type"]
                        dep_time = datetime.strptime(origin["time"],"%H:%M:%S")
                        arr_time = datetime.strptime(destination["time"],"%H:%M:%S")
                        dep_stations.append(origin["name"])
                        arr_stations.append(destination["name"])
                        if mode=="WALK":
                            walk_time = (arr_time-dep_time).total_seconds()//60
                            trans_times.append(walk_time)
                            dep_times.append(0)
                            arr_times.append(0)
                        else:
                            trans_times.append(0)
                            dep_times.append(dep_time)
                            arr_times.append(arr_time)
                            lines.append(int(answer_leg["name"].split(" ")[-1]))
                        departure_time = answer_leg["Destination"]
                    df_add = pd.DataFrame([[journey[0],journey[1],lines,dep_stations,arr_stations,dep_times,arr_times,trans_times,request_time]],columns=df_planner.columns)
                    df_planner = pd.concat([df_planner,df_add],ignore_index=True)
            except:
                print("Fail from "+journey[0]+" to "+journey[1])
        if not os.path.isfile('data/planner_'+day+'.csv'):
            df_planner.to_csv('data/planner_'+day+'.csv',
                      header='column_names', index=False, encoding='utf-8')
        else:  # else it exists so append without writing the header
            df_planner.to_csv('data/planner_'+day+'.csv', mode='a',
                      header=False, index=False, encoding='utf-8')
        sleep(30)
    else:
        sleep(10)
        day = str(datetime.today().date())


Bergshamra bus Buses 2023-05-11 16:00:14.424324
Globen Metros 2023-05-11 16:10:59.642279
Globen Trams 2023-05-11 16:10:59.642279
Bergshamra bus Buses 2023-05-11 16:10:59.751241
T-Centralen Metros 2023-05-11 16:12:20.408919
Tekniska högskolan Metros 2023-05-11 16:12:20.444552
Tekniska högskolan Buses 2023-05-11 16:12:20.445058
Solna Centrum Metros 2023-05-11 16:12:20.476418
Solna Centrum Buses 2023-05-11 16:12:20.476418
Solna Centrum Trams 2023-05-11 16:12:20.477220
Solna Bus Buses 2023-05-11 16:12:20.503344
T-Centralen Metros 2023-05-11 16:14:19.021059
Tekniska högskolan Metros 2023-05-11 16:14:19.057766
Tekniska högskolan Buses 2023-05-11 16:14:19.057766
Fridhemsplan Metros 2023-05-11 16:14:22.257657
Fridhemsplan Buses 2023-05-11 16:14:22.257657
Globen Metros 2023-05-11 16:14:22.303942
Globen Trams 2023-05-11 16:14:22.304935
Solna Bus Buses 2023-05-11 16:15:07.364950
Fridhemsplan Metros 2023-05-11 16:15:07.392983
Fridhemsplan Buses 2023-05-11 16:15:07.392983
Globen Metros 2023-05-11 1

ConnectionError: HTTPConnectionPool(host='api.sl.se', port=80): Max retries exceeded with url: /api2/TravelplannerV3_1/trip.csv?key=129060a9519144769f3e1891d9f96dd4&Lang=en&originId=9115&destId=9204 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000181A870E460>: Failed to establish a new connection: [WinError 10060] Une tentative de connexion a échoué car le parti connecté n’a pas répondu convenablement au-delà d’une certaine durée ou une connexion établie a échoué car l’hôte de connexion n’a pas répondu'))